In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import requests
from bs4 import BeautifulSoup, NavigableString, Tag, Comment

In [2]:
# !pip install opencv-python

In [3]:
from selenium import webdriver

driver_path = r"D:\yun\dev-tools\chromedriver.exe"
browser = webdriver.Chrome(executable_path=driver_path)

In [4]:
import pandas as pd

pr = pd.period_range('20201101', '20201119', freq='d')
print(pr)

PeriodIndex(['2020-11-01', '2020-11-02', '2020-11-03', '2020-11-04',
             '2020-11-05', '2020-11-06', '2020-11-07', '2020-11-08',
             '2020-11-09', '2020-11-10', '2020-11-11', '2020-11-12',
             '2020-11-13', '2020-11-14', '2020-11-15', '2020-11-16',
             '2020-11-17', '2020-11-18', '2020-11-19'],
            dtype='period[D]', freq='D')


In [7]:
url="https://news.naver.com/main/list.nhn"
query= "mode=LPOD&sid2=140&sid1=001&mid=sec&oid=001&isYeonhapFlash=Y&date={0}&page={1}"

day = pd.period_range('20201117', '20201119').strftime("%Y%m%d").tolist()
# day = ["20201118","20201119"]
page = 1
print(day)

['20201117', '20201118', '20201119']


In [8]:
news_list=[]

for idx,date in enumerate(day):
    for page in range(1,4):
        time.sleep(0.3)
        current_url = "".join([url, "?", query]) \
                        .format(date, page)
        # page += 1

        browser.get(current_url)    
        soup = BeautifulSoup(browser.page_source, "html.parser") 
        # print(soup)

        dl_list = soup.select("#main_content > div.list_body.newsflash_body > ul.type02 > li.")
        # print(dl_list)
        if len(dl_list) == 0:
            break
        #main_content > div.list_body.newsflash_body > ul > li:nth-child(2)
        for idx, dl in enumerate(dl_list):
            links = dl.select('a.nclicks(cnt_flashart)')    #링크
            date = dl.select('span.date')    #날짜
            # print(len(links))
            # print(links)
            # print(date)
            link = links[0] if len(links) == 1 else links[1]     
            # len(links) == 1 이면 links[0] , 아니면 links[1]
            date = date[0]
            news_list.append( [link.text.strip(), link['href'], date.text.strip()])

        if len(dl_list) < 3:
            break

        page += 1

In [9]:
len(news_list) 

27

In [11]:
# print(len(news_list[4:7]))
news_list[:3]

[['황희찬, 한국 A매치 역대 최단시간 16초 만에 득점 신기록',
  'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012021604&isYeonhapFlash=Y&rc=N',
  '2020.11.17. 오후 10:31'],
 ["산은, '아시아나 인수' 한진그룹 오너 갑질 막는다",
  'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012021173&isYeonhapFlash=Y&rc=N',
  '2020.11.17. 오후 6:44'],
 ['광화문 확장에…오세훈 "무리수" 안철수 "유훈행정"',
  'https://news.naver.com/main/read.nhn?mode=LPOD&mid=sec&oid=001&aid=0012021470&isYeonhapFlash=Y&rc=N',
  '2020.11.17. 오후 9:09']]

In [ ]:
# none_contents_index = []
# for idx, news in enumerate(news_list):

#     time.sleep(0.3)

#     link2 = news[1]
#     browser.get(link2)
#     soup2 = BeautifulSoup(browser.page_source, 'html.parser')
#     # print(soup2)
#     contents = soup2.select_one('div#articleBodyContents')
#     contents2= soup2.select_one('div#newsEndContents')
#     # print("current index : ", idx)
#     # print(contents2)
#     content_string = ""    
#     if (contents != None):
#         for child in contents.children:        
#             # print(child, '/', type(child))
#             if type(child) == Tag and child.name != "script":
#                 content_string += child.text.strip() + " " 
#             if type(child) == NavigableString:   
#                 content_string += str(child).strip() + " "
        
#         print(content_string)
#         news.append(content_string) # [제목,주소] -> [제목,주소,내용]
#     else:
#         none_contents_index.append(idx)

In [12]:
none_contents_index = []
for idx, news in enumerate(news_list):

    time.sleep(0.3)

    link2 = news[1]
    # print(link2)
    browser.get(link2)
    soup2 = BeautifulSoup(browser.page_source, 'html.parser')
    # # print(soup2)
    contents = soup2.select_one('div#articleBodyContents')
    contents2= soup2.select_one('div#newsEndContents')
    # # print("current index : ", idx)
    # # print(contents2)
    content_string = ""    
    if (contents != None):
        for child in contents.children:        
            # print(child, '/', type(child))
            if type(child) == Tag and child.name != "script":
                content_string += child.text.strip() + " " 
            if type(child) == NavigableString:   
                content_string += str(child).strip() + " "
        
        # print(content_string)
        news.append(content_string) # [제목,주소,날짜] -> [제목,주소,날짜,내용]
    elif (contents == None):
        for child in contents2.children:        
            # print(child, '/', type(child))
            if type(child) == Tag and child.name != "script":
                content_string += child.text.strip() + " " 
            if type(child) == NavigableString:   
                content_string += str(child).strip() + " "
        # print(content_string)
        news.append(content_string)
    else:
        none_contents_index.append(idx)

In [13]:
none_contents_index

[]

In [ ]:
news_list[7]

In [14]:
news_list_df = \
pd.DataFrame(news_list, columns=['제목', '링크', '날짜','내용',])

In [15]:
news_list_df

,제목,링크,날짜,내용
0,"황희찬, 한국 A매치 역대 최단시간 16초 만에 득점 신기록",https://news.naver.com/main/read.nhn?mode=LPOD...,2020.11.17. 오후 10:31,카타르와 평가전서 황의조 어시스트로 '선제 벼락골' 작성 황희찬 선제골(서울=...
1,"산은, '아시아나 인수' 한진그룹 오너 갑질 막는다",https://news.naver.com/main/read.nhn?mode=LPOD...,2020.11.17. 오후 6:44,한진칼과 투자합의서 체결…윤리경영 안전장치 마련위약금 5천억 등 7대 의무...
2,"광화문 확장에…오세훈 ""무리수"" 안철수 ""유훈행정""",https://news.naver.com/main/read.nhn?mode=LPOD...,2020.11.17. 오후 9:09,서울시 광화문광장 탈바꿈 공사 시작(서울=연합뉴스) 백승렬 기자 = 17일...
3,알테어 3점포 쾅…NC 완벽 계투로 한국시리즈 기선 제압,https://news.naver.com/main/read.nhn?mode=LPOD...,2020.11.17. 오후 9:55,두산 5-3으로 따돌려…5명 벌떼 구원진 3⅔이닝 무실점 알테어 홈런에 환호하...
4,"KBS, '검언유착' 오보 관련자 감봉 등 징계",https://news.naver.com/main/read.nhn?mode=LPOD...,2020.11.17. 오후 8:30,"1·3노조 ""솜방망이 처벌…지나치게 가벼운 징계"" KBS 검언유착오보진..."
5,손흥민-황의조-황희찬 '최정예 삼각편대' 카타르전 출격,https://news.naver.com/main/read.nhn?mode=LPOD...,2020.11.17. 오후 9:13,카타르전 축구대표팀 선발 명단 [대한축구협회 제공. 재판매 및 DB 금지] ...
6,전세대책 발표 19일로 연기…단기간에 공공임대 10만호 공급,https://news.naver.com/main/read.nhn?mode=LPOD...,2020.11.17. 오후 5:03,"민간 짓는 다가구주택, 서울시내 호텔과 사무실 등 주거용으로 바꿔 공급 ..."
7,"전문가들 ""거리두기 1.5단계 격상 필요""…일각선 실효성에 의문",https://news.naver.com/main/read.nhn?mode=LPOD...,2020.11.17. 오후 7:32,"""코로나19 확산세 차단에 어느 정도 효과있을지는 두고 봐야"" '거리..."
8,맨시티전 앞둔 토트넘 '골잡이' 손흥민 복귀에 전세기 투입,https://news.naver.com/main/read.nhn?mode=LPOD...,2020.11.17. 오후 8:49,손흥민 '브이' [대한축구협회 제공. 재판매 및 DB 금지] (서울=연합...
9,"모더나 이어 화이자도 ""코로나 백신 95% 면역효과""…최종 결과",https://news.naver.com/main/read.nhn?mode=LPOD...,2020.11.18. 오후 9:57,모더나 94.5% 중간결과 발표 이틀만…광범위한 백신 보급 기대감 높여 화...


In [16]:
news_list_df.to_csv('news_크롤링.csv')     # 데이터 csv로 저장

In [17]:
browser.close()

In [34]:
url="https://news.daum.net/breakingnews/society?page=3&regDate=20201001"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')

resp.status_code

200

In [52]:
div_list=soup.select("#mArticle > div.box_etc > ul > li >div")

news_list = []

if len(div_list) == 0:
    pass

for idx, div in enumerate(div_list):
    links = div.select('strong > a')
    link = links[0] if len(links) == 1 else links[1]
    news_list.append( [link.text.strip(), link['href']]) 

news_list[:5]

[["진천 요양원 코호트 격리 해제..16명 모두 '음성'", 'https://v.daum.net/v/20201001224638609'],
 ["성묘도 나들이도 '예약제'..방역수칙 지킨 추석 풍경", 'https://v.daum.net/v/20201001224635608'],
 ['보령 해상서 어선 침몰..인명피해 없어', 'https://v.daum.net/v/20201001224358576'],
 ['천안서 코로나19 확진자 1명 추가', 'https://v.daum.net/v/20201001224357575'],
 ['[포토뉴스] 안병용 의정부시장 부용시민정원 식재', 'https://v.daum.net/v/20201001224303565']]

In [2]:
url="https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EB%A6%AC&tbm=isch&ved=2ahUKEwjVjaeY3dTsAhUFfpQKHS2OBigQ2-cCegQIABAA&oq=%ED%8C%8C%EC%9D%B4%EB%A6%AC&gs_lcp=CgNpbWcQAzIFCAAQsQMyAggAMgIIADICCAAyAggAMgIIADICCAAyAggAMgIIADICCAA6BAgjECdQmau8AVjmtLwBYPLMvAFoAXAAeAKAAfIGiAHTEpIBDTAuMy4xLjAuMi4wLjGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=YwyYX5XQIIX80QStnJrAAg&bih=937&biw=1903&hl=ko"

resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')

resp.status_code

200